# OBTNER DATOS DEL PROTOTIPO

In [ ]:


%matplotlib inline

#Varaibles globales
VentanaTiempo = 5 
TiempoFinal = 10

# Iniciamos comunicación serial
ser = serial.Serial('/COM9', 256000)
ser.flushInput()

#0-10 muestras corresponde a 100 segundos
for p in range(0,10):
    
    
    # Lista para guardar datos Tiempo y Amplitud
    timepoints = []
    ydata = []
    tiempo1=[]
    run1 = True

    #validamos que un dato ECG >=800
    while run1:
        ser.reset_input_buffer()
        data = str(ser.readline()) # Lectura del puerto
        
        # El envío se realiza con un marcador $, sólo si la trama tiene ese marcador se separará los datos de la trama
        ecg=data.split('#')[1].split('\\r\\n')[0]
        
        if float(ecg)>=800:
            run2=True
            print("ESPERE TOMANDO MUESTRA")
            run1=False

    ##inicia el time
    start_time = time()
    while run2:

        #ser.reset_input_buffer()
        data = str(ser.readline()) # Lectura del puerto
        # El envío se realiza con un marcador $, sólo si la trama tiene ese marcador se separará los datos de la trama
        ecg=data.split('#')[1].split('\\r\\n')[0]

        # Agregamos los datos de amplitud y tiempo a cada lista
        ydata.append(float(ecg))
        timepoints.append(time()-start_time)
        pos=pos+1
        
        current_time = timepoints[-1]
       
        # La ejecución termina cuando el tiempo de ejecución llega al límite
        if timepoints[-1] > TiempoFinal: 
            run2=False
    
    #GRAFICAMOS LOS DATOS
    tiempo=(timepoints[len(timepoints)-1]-timepoints[0])
    sum=0
    #tomando muestra de los primeros 100 datos
    for i in range(1,101):
        tiempo1.append(timepoints[i]-timepoints[i-1])
    #sumatoria de los tiempos(100 datos)
    for m in range(0,len(tiempo1)):
        sum=sum+tiempo1[m]
    #tiempo promedio o periodo  
    t_muestreo_prom=sum/len(tiempo1)
    print("PERIODO: ", t_muestreo_prom)
    #tiempo total de la muestra
    print("TIEMPO MUESTRA TOTAL: ", tiempo)

    #ser.close()    

    #visulizacion offline
    plt.suptitle('Gráfica de datos obtenidos '+str(p), fontsize='18', fontweight='bold')
    plt.xlabel('Tiempo (s)', fontsize='14')
    plt.ylabel('Amplitud', fontsize='14')
    plt.axes().grid(True)
    plt.plot(timepoints, ydata,'r',marker='+',markersize=4,linestyle='-')
    plt.show()

    #GUARDAR DATOS
    df = pd.DataFrame(ydata) 
    df.to_csv('DATOS-MONITOR-TEST/test/pulso irregular/premature-atrial-cont#2/datos_Ecg'+str(p)+'.csv',header=None,index=False)
ser.close()    